In [11]:
from pulp import *
import numpy as np
import chess
from chess import svg
import io

def print_chess(chessBoard):
    if(len(chessBoard[0]) == 8): #If chessboard is 8x8, the fen representation is used for creating high quality chess board.The output file is in SVG format.

        for a in range (0,len(chessBoard)):
            for b in range(0,len(chessBoard[a])):
                if chessBoard[a][b] == 0.0:
                    chessBoard[a][b] = 'en'
                else:
                    chessBoard[a][b] = 'bq'
        svgs = board_to_fen(chessBoard)
        board = chess.Board(svgs)
        boardsvg = chess.svg.board(board = board)
        filetowriteto = open("chess.SVG", "w")
        filetowriteto.write(boardsvg)
        filetowriteto.close()
    else: #If chessboard is not 8x8, then board is printed using print statement. "." indicates empty square and "Q" indicate queen.
        for a in range(0,len(chessBoard)):
            for b in range(0,len(chessBoard[a])):
                if chessBoard[a][b] == 0.0:
                    print(".",end = " ")
                else:
                    print("Q", end = " ")
            print("\n")

    

In [12]:
 def board_to_fen(board):
        '''
        This function takes 8x8 array as an input and returns fen representation. 
        If array does not corresponds to chess representation
        function fails.
        
        Usage board_to_fen(chess_board)
        return string
        '''
        
        # Use StringIO to build string more efficiently than concatenating
        with io.StringIO() as s:
            for row in board:
                empty = 0
                for cell in row:
                    c = cell[0]
                    if c in ('w', 'b'):
                        if empty > 0:
                            s.write(str(empty))
                            empty = 0
                        s.write(cell[1].upper() if c == 'w' else cell[1].lower())
                    else:
                        empty += 1
                if empty > 0:
                    s.write(str(empty))
                s.write('/')
            # Move one position back to overwrite last '/'
            s.seek(s.tell() - 1)
            # If you do not have the additional information choose what to put
            s.write(' w KQkq - 0 1')
            return s.getvalue()

In [16]:
def nQueensProblem(n):
    '''
    This function uses 0-1 integer linear programming algorithm to solve n-queens problem. If solution is optimal
    it is printed out. There is a special case for n = 8. Rather than printing out the solution, SVG file of chessboard is cre-
    ated in main directory.
    
    Linear programming is implemented using Pulp package in python.
    
    Usage nQueensProblem(N)
    
    returns 
    if n = 8:
        SVG file corresponding to chessboard array.
    else:
        printing out the chessboard array.
    '''
    N = n
    seq = [a for a in range(1,N+1)]
    columns = seq
    rows = seq
    prob = LpProblem("8-Queens Problem",LpMinimize)
    choices = LpVariable.dicts("Choices",(columns,rows),0,1,LpInteger)
    prob += 13
    for r in rows:
        #Every row has to contain 1 queen.
        a = lpSum([choices[r][c] for c in columns]) == 1 
        prob += a
        #right to left Diagonal for left-most column can contain
        # not more than 1 queen.
        a = lpSum([choices[r+m][1+m] for m in range(1,4-r+1)]) <= 1
        prob+= choices[r][1] + a
        #left to right Diagonal for right-most column can contain
        # not more than 1 queen.
        a = lpSum([choices[r+m][N-m] for m in range(1,N-r+1)]) <= 1
        prob+= choices[r][N] + a
    for c in columns:
        #Every column has to contain 1 queen.
        a = lpSum([choices[r][c] for r in rows]) == 1 
        prob += a
        #right to left Diagonal for the top column can contain
        # not more than 1 queen.
        a = lpSum([choices[1+m][c-m] for m in range(1,c)]) <= 1
        prob+= choices[1][c] + a
        #Left to right Diagonal for the top column can contain
        # not more than 1 queen.
        a = lpSum([choices[1+m][c+m] for m in range(1,4-c+1)]) <= 1
        prob+= choices[1][c] + a
 
    prob.solve()
    print ("Status:",LpStatus[prob.status])
    import chess
    chessBoard = np.zeros((N,N))
    #Setting the values of chessboard from the solution.
    for c in columns:
        for r in rows:
            if value(choices[c][r]) == 1:
                chessBoard[c-1][r-1] = 1 
    chessBoard = chessBoard.tolist()
    print_chess(chessBoard)

        

In [18]:
nQueensProblem(4)

Status: Optimal
. Q . . 

. . . Q 

Q . . . 

. . Q . 



C:\Users\akbab\Anaconda3\lib\site-packages\pulp\pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
